<a href="https://colab.research.google.com/github/parismoscow/Alzheimer-s_Disease/blob/master/Alzheimers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [162]:
!pip install imblearn

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE


In [0]:
df = pd.read_csv("/content/drive/My Drive/measures.csv")

In [165]:
df.shape[0]

12741

Use all suggested features

In [0]:
df = df.replace(r'^\s*$', np.nan, regex=True)
df.dropna(inplace=True)
df.loc[df['ABETA_UPENNBIOMK9_04_19_17'] == '<200', ]['ABETA_UPENNBIOMK9_04_19_17'] = 199
df['ABETA_UPENNBIOMK9_04_19_17'] = df['ABETA_UPENNBIOMK9_04_19_17'].astype(float)
df.loc[df['TAU_UPENNBIOMK9_04_19_17'] == '>1300', 'TAU_UPENNBIOMK9_04_19_17'] = 1301
df.loc[df['TAU_UPENNBIOMK9_04_19_17'] == '<80', 'TAU_UPENNBIOMK9_04_19_17'] = 79
df['TAU_UPENNBIOMK9_04_19_17'] = df['TAU_UPENNBIOMK9_04_19_17'].astype(float)
df.loc[df['PTAU_UPENNBIOMK9_04_19_17'] == '>120', 'PTAU_UPENNBIOMK9_04_19_17'] = 121
df.loc[df['PTAU_UPENNBIOMK9_04_19_17'] == '<8', 'PTAU_UPENNBIOMK9_04_19_17'] = 7
df['PTAU_UPENNBIOMK9_04_19_17'] = df['PTAU_UPENNBIOMK9_04_19_17'].astype(float)
X=df.drop(columns='DX')
y = df['DX']
# X_train, X_test, y_train, y_test = train_test_split (X, y, random_state=42)

In [206]:
df['DX'].count()

837

In [207]:
df['DX'].value_counts()

MCI                425
NL                 281
Dementia           107
MCI to Dementia     15
NL to MCI            5
MCI to NL            3
Dementia to MCI      1
Name: DX, dtype: int64

Remove diagnoses that are borderline

In [0]:
df = df.loc[(df['DX'] == 'MCI') | (df['DX'] == 'Dementia') | (df['DX'] == 'NL')]

In [242]:
df['DX'].value_counts()

MCI         425
NL          281
Dementia    107
Name: DX, dtype: int64

In [210]:
df['DX'].count()

813

In [0]:
y = df["DX"]
X = df.drop(columns='DX')

Use subset of suggested features


In [173]:
# testdf = df.drop(columns=['ABETA_UPENNBIOMK9_04_19_17', 'TAU_UPENNBIOMK9_04_19_17', 'PTAU_UPENNBIOMK9_04_19_17', 'FDG', 'AV45'])
testdf = df[['CDRSB', 'ADAS13', 'MMSE', 'ADAS11', 'RAVLT_immediate', 'DX']]

testdf.dropna(inplace=True)
testdf.shape[0]
# testdf
testdf.dtypes



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


CDRSB              float64
ADAS13             float64
MMSE               float64
ADAS11             float64
RAVLT_immediate    float64
DX                  object
dtype: object

In [0]:
# testdf.columns

In [175]:
testdf['DX'].value_counts()

MCI         425
NL          281
Dementia    107
Name: DX, dtype: int64

In [0]:
# y = testdf["DX"]
# X=testdf.drop(columns='DX')


Split data into train and test

In [0]:
X_train, X_test, y_train, y_test = train_test_split (X, y, random_state=42)

Scale the features

In [0]:
X_minmax = MinMaxScaler().fit(X_train)

X_train_s = X_minmax.transform(X_train)
X_test = X_minmax.transform(X_test)

# X_scaler = StandardScaler().fit(X_train)
# X_train = X_scaler.transform(X_train)
# X_test = X_scaler.transform(X_test)

Ovesample Demential and NL samples with SMOTE


In [246]:
y_train.value_counts()

MCI         317
NL          210
Dementia     82
Name: DX, dtype: int64

In [0]:
smt = SMOTE()
X_train_s, y_train_s = smt.fit_sample(X_train, y_train)

In [248]:
from collections import Counter

print('Resampled dataset shape %s' % Counter(y_train_s))
# np.bincount(y_train)

Resampled dataset shape Counter({'MCI': 317, 'NL': 317, 'Dementia': 317})


SVC model

In [218]:
from sklearn.svm import SVC
model = SVC(kernel='linear')
model.fit(X_train_s, y_train_s)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [219]:
model.score(X_test, y_test)

0.553921568627451

In [0]:
predictions = model.predict(X_test)

In [184]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[21,  4,  0],
       [ 4, 78, 26],
       [ 0,  2, 69]])

In [185]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

    Dementia       0.84      0.84      0.84        25
         MCI       0.93      0.72      0.81       108
          NL       0.73      0.97      0.83        71

    accuracy                           0.82       204
   macro avg       0.83      0.84      0.83       204
weighted avg       0.85      0.82      0.82       204



Random Forest

In [249]:
rf = RandomForestClassifier(n_estimators=100)
rf = rf.fit(X_train_s, y_train_s)
rf.score(X_test, y_test)


0.9264705882352942

In [237]:
predictions = rf.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

    Dementia       0.84      0.84      0.84        25
         MCI       0.93      0.94      0.94       108
          NL       0.97      0.94      0.96        71

    accuracy                           0.93       204
   macro avg       0.91      0.91      0.91       204
weighted avg       0.93      0.93      0.93       204



In [238]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[ 21,   4,   0],
       [  4, 102,   2],
       [  0,   4,  67]])

In [0]:
importances = rf.feature_importances_


In [0]:
# X2_features = X2.columns
X_features = X.columns

In [227]:
sorted(zip(rf.feature_importances_, X_features), reverse=True)

[(0.37900453352316577, 'CDRSB'),
 (0.12358290715708806, 'ADAS13'),
 (0.11095800924999923, 'ADAS11'),
 (0.10351035621031743, 'MMSE'),
 (0.0531776560334385, 'RAVLT_immediate'),
 (0.03415291025428216, 'FDG'),
 (0.026969593016989524, 'ABETA_UPENNBIOMK9_04_19_17'),
 (0.024816978469119233, 'AGE'),
 (0.023380120378983373, 'Hippocampus'),
 (0.022301264288228175, 'AV45'),
 (0.018297423632261948, 'Entorhinal'),
 (0.016504000603252866, 'PTAU_UPENNBIOMK9_04_19_17'),
 (0.015584370581552341, 'Ventricles'),
 (0.014425634907506073, 'WholeBrain'),
 (0.013421375801342825, 'TAU_UPENNBIOMK9_04_19_17'),
 (0.012210173102571744, 'MidTemp'),
 (0.007702692789900871, 'APOE4')]

UAC 


In [0]:
# import numpy as np
# import matplotlib.pyplot as plt
# from itertools import cycle

# from sklearn import svm, datasets
# from sklearn.metrics import roc_curve, auc
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import label_binarize
# from sklearn.multiclass import OneVsRestClassifier
# from scipy import interp

# # Import some data to play with
# # iris = datasets.load_iris()
# # X = iris.data
# # y = iris.target

# # Binarize the output
# y = label_binarize(y, classes=[0, 1, 2])
# n_classes = y.shape[1]

# # Add noisy features to make the problem harder
# # random_state = np.random.RandomState(0)
# # n_samples, n_features = X.shape
# # X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

# # shuffle and split training and test sets
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5,
# #                                                     random_state=0)

# # # Learn to predict each class against the other
# # classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True,
# #                                  random_state=random_state))
# y_score = classifier.fit(X_train, y_train).decision_function(X_test_s)

# # Compute ROC curve and ROC area for each class
# fpr = dict()
# tpr = dict()
# roc_auc = dict()
# for i in range(n_classes):
#     fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
#     roc_auc[i] = auc(fpr[i], tpr[i])

# # Compute micro-average ROC curve and ROC area
# fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
# roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [0]:
# from sklearn.dummy import DummyClassifier
# dummy = DummyClassifier(strategy="most_frequent")
# dummy.fit(X_train_s, y_train_s)
# dummy.score(X_test, y_test)
# predictions = dummy.predict(X_test)
# print(classification_report(y_test, predictions))

In [0]:
# 